In [56]:
from itertools import product
from random import random, randint, seed, uniform, sample
import numpy as np
import math
from scipy import sparse
from copy import copy
from tqdm import tqdm
from functools import reduce
from collections import deque

In [57]:
def make_set_covering_problem(num_points, num_sets, density, prob=None):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool).toarray()
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    if prob is not None:
        np.random.seed(int(num_points * 435761 + num_sets + density))
        initial_state = np.random.choice([True, False], size=(num_sets,), p=[prob, 1 - prob])
    else:
        initial_state = np.full((num_sets,), False, dtype=np.bool_)
    return sets, initial_state

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

## Problem definition

In [58]:
NUM_SETS = 1000
NUM_POINTS = 1000
DENSITY = 0.3
# PROB = 0.02

# problem, initial_state = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY, PROB)
problem, initial_state = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
print(
    f'Problem shape: {problem.shape}',
    f'Initial state shape: {initial_state.shape}, Taken sets: {np.sum(initial_state)}',
    sep='\n',
)

Problem shape: (1000, 1000)
Initial state shape: (1000,), Taken sets: 0


In [59]:
def covered(problem, state):
    return reduce(
        np.logical_or,
        [problem[i] for i, t in enumerate(state) if t],
        np.array([False for _ in range(NUM_POINTS)]),
    )


def check_goal(problem, state):
    return np.all(covered(problem, state))

In [61]:
assert check_goal(problem, np.full((NUM_SETS,), True, dtype=np.bool_)), "Problem not solvable"

In [60]:
def fitness1(problem, state):
    goal = check_goal(problem, state)
    cost = np.sum(state)
    return goal, cost if not goal else -cost


def fitness2(problem, state):
    goal = np.sum(covered(problem, state))
    cost = sum(state)
    return goal, -cost


fitness = fitness2

In [62]:
fitness(problem, initial_state)

(0, 0)

## Implemented methods - Single-state methods
The Halloween challenge has been accepted. To solve it, I'd reply with the following methods (with some modifications described for each algorithm, if any):
- **Random-Mutation Hill Climber**;
- **Steepest-Step Hill Climber**;
- **Steepest-Step Hill Climber with Replacement**;
- **Tabu Search**;
- **Simulated Annealing**.

In [63]:
def random_tweak(state):
    """
    Tweak a state randomly.

    Args:
        state: 1-D boolean ndarray.

    Returns:
        New state with a changed boolean value.
    """
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state


def tweak_by_index(state, index):
    """
    Tweak a state changing the value in position index.

    Args:
        state: 1-D boolean ndarray;
        index: int value indicating the boolean value to change.

    Returns:
        New state with a changed boolean value according
        to the value of index.
    """
    new_state = copy(state)
    new_state[index] = not new_state[index]
    return new_state

To make the Random-Mutation Hill Climber run faster, I decided to give up the search when for `max_give_up` times I do not improve my best current solution.

In [64]:
def RMHC(problem, state, fitness, max_it, max_give_up):
    """
    Random-Mutation Hill Climber implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        max_it: maximum number of iterations (int);
        max_give_up: maximum number of evaluations before giving up (int).

    Returns:
        Possible state solution to the problem.
    """
    changes = 0
    evals_giveup = 0
    evals = 1
    best_fitness = fitness(problem, state)
    for it in tqdm(range(max_it)):
        new_state = random_tweak(state)
        evals += 1
        new_state_fitness = fitness(problem, new_state)
        if new_state_fitness > best_fitness:
            state = new_state
            best_fitness = new_state_fitness
            changes += 1
            evals_giveup = 0
        else:
            evals_giveup += 1
            if evals_giveup == max_give_up:
                print('Maximum number of evaluations without improvement reached.')
                break

    it += 1
    if it == max_it:
        print('Maximum number of iterations reached.')
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = best_fitness
    cond = goal if isinstance(goal, np.bool_) else goal == problem.shape[1]

    print(f'Goal reached? {"Yes" if cond else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [11]:
_ = RMHC(problem, initial_state, fitness, 10_000, 2_000)

 26%|██▋       | 2641/10000 [00:00<00:01, 5572.68it/s]

Maximum number of evaluations without improvement reached.
Terminated after 2642 iterations.
Terminated after 26 changes.
Number of evaluations: 2643.
Goal reached? Yes
State cost: 14


In [65]:
def SAHC(problem, state, fitness, max_it):
    """
    Steepest-Ascent Hill Climber implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        max_it: maximum number of iterations (int).

    Returns:
        Possible state solution to the problem.
    """
    changes = 0
    evals = 1
    best_fitness = fitness(problem, state)
    for it in tqdm(range(max_it)):
        succ = tweak_by_index(state, 0)
        evals += 1
        succ_fitness = fitness(problem, succ)
        for index in range(1, NUM_SETS):
            new_state = tweak_by_index(state, index)
            evals += 1
            new_state_fitness = fitness(problem, new_state)
            if new_state_fitness > succ_fitness:
                succ = new_state
                succ_fitness = new_state_fitness
        if succ_fitness > best_fitness:
            state = succ
            best_fitness = succ_fitness
            changes += 1
        else:
            break

    it += 1
    if it == max_it:
        print('Maximum number of iterations reached.')
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = best_fitness
    cond = goal if isinstance(goal, np.bool_) else goal == problem.shape[1]

    print(f'Goal reached? {"Yes" if cond else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [13]:
_ = SAHC(problem, initial_state, fitness, 1_000)

  1%|          | 10/1000 [00:01<03:05,  5.34it/s]

Terminated after 11 iterations.
Terminated after 10 changes.
Number of evaluations: 11001.
Goal reached? Yes
State cost: 10


In [66]:
def SAHCwReplacement(problem, state, fitness, n_neighbors, max_it):
    """
    Steepest-Ascent Hill Climber implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        n_neighbors: number of desired tweaks to try;
        max_it: maximum number of iterations (int).

    Returns:
        Possible state solution to the problem.
    """
    changes = 0
    evals = 1
    best_fitness = fitness(problem, state)
    for it in tqdm(range(max_it)):
        index = randint(0, NUM_SETS - 1)
        succ = tweak_by_index(state, index)
        evals += 1
        succ_fitness = fitness(problem, succ)
        for index in set(sample(range(NUM_SETS), n_neighbors)) - {index}:
            new_state = tweak_by_index(state, index)
            evals += 1
            new_state_fitness = fitness(problem, new_state)
            if new_state_fitness > succ_fitness:
                succ = new_state
                succ_fitness = new_state_fitness
        if succ_fitness > best_fitness:
            state = succ
            best_fitness = succ_fitness
            changes += 1
        else:
            break

    it += 1
    if it == max_it:
        print('Maximum number of iterations reached.')
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = best_fitness
    cond = goal if isinstance(goal, np.bool_) else goal == problem.shape[1]

    print(f'Goal reached? {"Yes" if cond else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [22]:
_ = SAHCwReplacement(problem, initial_state, fitness, 100, 1_000)

  1%|          | 11/1000 [00:00<00:19, 51.33it/s]

Terminated after 12 iterations.
Terminated after 11 changes.
Number of evaluations: 1211.
Goal reached? Yes
State cost: 11


For the following method, I took suggestions and code parts from these links:
- [www.geeksforgeeks.org](https://www.geeksforgeeks.org/what-is-tabu-search/);
- [stackoverflow.com](https://stackoverflow.com/questions/60492520/check-if-array-is-in-deque-of-arrays-python)

I implemented the method in a slightly different way than what you can find on the first link. I stop the search if the best successor is not better than the best current solution or every successor is already in the tabu list.

In [67]:
def tabu_search(problem, state, fitness, n_neighbors, max_tabu_list, max_it):
    """
    Steepest-Ascent Hill Climber implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        n_neighbors: number of desired tweaks to try;
        max_tabu_list: tabu list size;
        max_it: maximum number of iterations (int).

    Returns:
        Possible state solution to the problem.
    """
    changes = 0
    evals = 1
    best_fitness = fitness(problem, state)
    tabu_list = deque()
    for it in tqdm(range(max_it)):
        index = randint(0, NUM_SETS - 1)
        succ = tweak_by_index(state, index)
        evals += 1
        succ_fitness = fitness(problem, succ)
        for index in set(sample(range(NUM_SETS), n_neighbors)) - {index}:
            new_state = tweak_by_index(state, index)
            if not any((new_state == elem).all() for elem in tabu_list):
                evals += 1
                new_state_fitness = fitness(problem, new_state)
                if new_state_fitness > succ_fitness:
                    succ = new_state
                    succ_fitness = new_state_fitness
        if any((succ == elem).all() for elem in tabu_list):
            break
        tabu_list.append(succ)
        if len(tabu_list) > max_tabu_list:
            tabu_list.popleft()
        if succ_fitness > best_fitness:
            state = succ
            best_fitness = succ_fitness
            changes += 1
        else:
            break

    it += 1
    if it == max_it:
        print('Maximum number of iterations reached.')
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = best_fitness
    cond = goal if isinstance(goal, np.bool_) else goal == problem.shape[1]

    print(f'Goal reached? {"Yes" if cond else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [96]:
_ = tabu_search(problem, initial_state, fitness, 100, 5, 1_000)

  1%|          | 10/1000 [00:00<00:20, 48.68it/s]

Terminated after 11 iterations.
Terminated after 10 changes.
Number of evaluations: 1112.
Goal reached? Yes
State cost: 10


For the implementation of the Simulated Annealing algorithm, I tried to create a function which gets as input a vector of floating point numbers (also called _schedule_), that represents a mapping from the time $t$ to the temperature $T$. \
In this way, I couldn't find a good vector for the _schedule_. I tried to generate several vectors by means of the `np.linspace` function, but none of them worked successfully enough to find an acceptable solution.

For this reason, I surfed over the internet to implement an alternative approach without the use of this `schedule` vector. \
The solution I implemented is based on the pseudocode available on [en.wikipedia.org](https://en.wikipedia.org/wiki/Simulated_annealing), which calculates the temperature $T$ directly from the iteration number `it` and the maximum number of iterations `max_it`.

If the value `max_it` is large enough, the instructions `math.exp((new_state_fitness[1] - best_fitness[1]) / T)` and `np.exp((new_state_fitness[1] - best_fitness[1]) / T)` return an overflow error. \
To avoid this, you should convert `(new_state_fitness[1] - best_fitness[1]) / T)` to `np.float128`. Unfortunately, this numpy data type is not available on my system.

If you do this data type conversion, you should be able to find a much better solution.

In [68]:
def simulated_annealing(problem, state, fitness, max_it):
    """
    Simulated Annealing implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        max_it: maximum number of iterations (int).

    Returns:
        Possible state solution to the problem.
    """
    changes = 0
    evals = 1
    best_fitness = fitness(problem, state)
    for it in tqdm(range(max_it)):
        T = 1 - ((it + 1) / max_it)
        if T == 0:
            break
        new_state = random_tweak(state)
        evals += 1
        new_state_fitness = fitness(problem, new_state)
        if new_state_fitness > best_fitness:
            state = new_state
            best_fitness = new_state_fitness
            changes += 1
        else:
            if uniform(0, 1) < math.exp((new_state_fitness[1] - best_fitness[1]) / T):
                state = new_state
                best_fitness = new_state_fitness
                changes += 1

    it += 1
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = best_fitness
    cond = goal if isinstance(goal, np.bool_) else goal == problem.shape[1]

    print(f'Goal reached? {"Yes" if cond else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [67]:
_ = simulated_annealing(problem, initial_state, fitness, 5_400)

100%|█████████▉| 5399/5400 [00:01<00:00, 4177.21it/s]

Terminated after 5400 iterations.
Terminated after 1301 changes.
Number of evaluations: 5400.
Goal reached? Yes
State cost: 19


## Results

### `fitness = fitness1`

In [74]:
fitness = fitness1

for NUM_POINTS, DENSITY in product((100, 1_000, 5_000), (0.3, 0.7)):
    NUM_SETS = NUM_POINTS

    print(f'** Combination: NUM_POINTS={NUM_POINTS}, NUM_SETS={NUM_SETS}, DENSITY={DENSITY} **', end='\n\n')

    problem, initial_state = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
    print(
        f'Problem shape: {problem.shape}',
        f'Initial state shape: {initial_state.shape}, Taken sets: {np.sum(initial_state)}',
        sep='\n',
    )

    print('\n-- RMHC')
    _ = RMHC(problem, initial_state, fitness, 10_000, 2_000)
    print('\n-- SAHC')
    _ = SAHC(problem, initial_state, fitness, 1_000)
    print('\n-- SAHCwReplacement')
    _ = SAHCwReplacement(problem, initial_state, fitness, 100, 1_000)
    print('\n-- tabu_search')
    _ = tabu_search(problem, initial_state, fitness, 100, 5, 1_000)
    print('\n-- simulated_annealing')
    _ = simulated_annealing(problem, initial_state, fitness, 905)

    print()

** Combination: NUM_POINTS=100, NUM_SETS=100, DENSITY=0.3 **

Problem shape: (100, 100)
Initial state shape: (100,), Taken sets: 0

-- RMHC


 22%|██▏       | 2183/10000 [00:00<00:00, 43350.59it/s]


Maximum number of evaluations without improvement reached.
Terminated after 2184 iterations.
Terminated after 14 changes.
Number of evaluations: 2185.
Goal reached? Yes
State cost: 8

-- SAHC


  1%|          | 12/1000 [00:00<00:02, 492.59it/s]


Terminated after 13 iterations.
Terminated after 12 changes.
Number of evaluations: 1301.
Goal reached? Yes
State cost: 10

-- SAHCwReplacement


...

Terminated after 7 iterations.
Terminated after 6 changes.
Number of evaluations: 708.
Goal reached? Yes
State cost: 6

-- tabu_search


  1%|          | 6/1000 [00:00<00:58, 16.98it/s]


Terminated after 7 iterations.
Terminated after 6 changes.
Number of evaluations: 707.
Goal reached? Yes
State cost: 6

-- simulated_annealing


100%|█████████▉| 904/905 [00:00<00:00, 1755.36it/s]

Terminated after 905 iterations.
Terminated after 147 changes.
Number of evaluations: 905.
Goal reached? Yes
State cost: 115



### `fitness = fitness2`

In [24]:
fitness = fitness2

for NUM_POINTS, DENSITY in product((100, 1_000, 5_000), (0.3, 0.7)):
    NUM_SETS = NUM_POINTS

    print(f'** Combination: NUM_POINTS={NUM_POINTS}, NUM_SETS={NUM_SETS}, DENSITY={DENSITY} **', end='\n\n')

    problem, initial_state = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
    print(
        f'Problem shape: {problem.shape}',
        f'Initial state shape: {initial_state.shape}, Taken sets: {np.sum(initial_state)}',
        sep='\n',
    )

    print('\n-- RMHC')
    _ = RMHC(problem, initial_state, fitness, 10_000, 2_000)
    print('\n-- SAHC')
    _ = SAHC(problem, initial_state, fitness, 1_000)
    print('\n-- SAHCwReplacement')
    _ = SAHCwReplacement(problem, initial_state, fitness, 100, 1_000)
    print('\n-- tabu_search')
    _ = tabu_search(problem, initial_state, fitness, 100, 5, 1_000)
    print('\n-- simulated_annealing')
    _ = simulated_annealing(problem, initial_state, fitness, 5_400)

    print()

** Combination: NUM_POINTS=100, NUM_SETS=100, DENSITY=0.3 **

Problem shape: (100, 100)
Initial state shape: (100,), Taken sets: 0

-- RMHC


 22%|██▏       | 2183/10000 [00:00<00:00, 30989.11it/s]


Maximum number of evaluations without improvement reached.
Terminated after 2184 iterations.
Terminated after 14 changes.
Number of evaluations: 2185.
Goal reached? Yes
State cost: 8

-- SAHC


  1%|          | 6/1000 [00:00<00:02, 337.57it/s]


Terminated after 7 iterations.
Terminated after 6 changes.
Number of evaluations: 701.
Goal reached? Yes
State cost: 6

-- SAHCwReplacement


...

Terminated after 6 iterations.
Terminated after 5 changes.
Number of evaluations: 607.
Goal reached? Yes
State cost: 5

-- tabu_search


  1%|          | 6/1000 [00:00<01:38, 10.10it/s]


Terminated after 7 iterations.
Terminated after 6 changes.
Number of evaluations: 707.
Goal reached? Yes
State cost: 6

-- simulated_annealing


100%|█████████▉| 5399/5400 [00:06<00:00, 890.24it/s]

Terminated after 5400 iterations.
Terminated after 1183 changes.
Number of evaluations: 5400.
Goal reached? Yes
State cost: 325



#### Run Simulated Annealing again with same arguments for both `fitness1` and `fitness2`

In [54]:
for NUM_POINTS, DENSITY in product((100, 1_000, 5_000), (0.3, 0.7)):
    NUM_SETS = NUM_POINTS

    print(f'** Combination: NUM_POINTS={NUM_POINTS}, NUM_SETS={NUM_SETS}, DENSITY={DENSITY} **', end='\n\n')

    problem, initial_state = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
    print(
        f'Problem shape: {problem.shape}',
        f'Initial state shape: {initial_state.shape}, Taken sets: {np.sum(initial_state)}',
        sep='\n',
    )

    print('\n-- simulated_annealing with fitness1')
    _ = simulated_annealing(problem, initial_state, fitness1, 905)
    print('\n-- simulated_annealing with fitness2')
    _ = simulated_annealing(problem, initial_state, fitness2, 905)

    print()

** Combination: NUM_POINTS=100, NUM_SETS=100, DENSITY=0.3 **

Problem shape: (100, 100)
Initial state shape: (100,), Taken sets: 0

-- simulated_annealing with fitness1


100%|█████████▉| 904/905 [00:00<00:00, 31919.75it/s]


Terminated after 905 iterations.
Terminated after 318 changes.
Number of evaluations: 905.
Goal reached? Yes
State cost: 14

-- simulated_annealing with fitness2


100%|█████████▉| 904/905 [00:00<00:00, 29199.79it/s]


Terminated after 905 iterations.
Terminated after 299 changes.
Number of evaluations: 905.
Goal reached? Yes
State cost: 9

** Combination: NUM_POINTS=100, NUM_SETS=100, DENSITY=0.7 **

Problem shape: (100, 100)
Initial state shape: (100,), Taken sets: 0

-- simulated_annealing with fitness1


...

Terminated after 905 iterations.
Terminated after 156 changes.
Number of evaluations: 905.
Goal reached? Yes
State cost: 120

** Combination: NUM_POINTS=5000, NUM_SETS=5000, DENSITY=0.7 **

Problem shape: (5000, 5000)
Initial state shape: (5000,), Taken sets: 0

-- simulated_annealing with fitness1


100%|█████████▉| 904/905 [00:00<00:00, 1783.70it/s]


Terminated after 905 iterations.
Terminated after 167 changes.
Number of evaluations: 905.
Goal reached? Yes
State cost: 123

-- simulated_annealing with fitness2


100%|█████████▉| 904/905 [00:00<00:00, 1104.17it/s]

Terminated after 905 iterations.
Terminated after 142 changes.
Number of evaluations: 905.
Goal reached? Yes
State cost: 112

